In [15]:
import sys
!{sys.executable} -m pip install lyricsgenius
!{sys.executable} -m pip install spotipy
!{sys.executable} -m pip install selenium
import pandas as pd
import requests
import lyricsgenius as genius
import spotipy
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [147]:
genius_client_id = 'Qeii-UKMUUwsAYCuuemsF7CZk7R2bFDhjP1FMkqpozASvYsA4wQDlLmd2JFM3Vhm'
genius_secret_id = 'fWhV7VtwAL-5D32Ga9Y0Kyh_VFEo7XK0Jf8-3RMcZ5EoMXqsDpmFWFFPyobnibfMXJuJ-GZe69YEuXR-36VGAA'
genius_client_access_token = '1QVL91jZbgdOIhvYhqKku_zkFDf6MHEyLpXEITHU2tE2HGvAqYIPInTrQaWcInD7'
app_website_url = 'http://cis545genius.com'
redirect_uri = 'cis545genius.com'
base_url = 'https://api.genius.com'
# driver_path = '/Users/ericdong/Downloads/chromedriver'
driver_path = "/mnt/c/Users/jeffr/Downloads/chromedriver_win32/chromedriver.exe"
artist_url = 'https://www.ranker.com/crowdranked-list/the-greatest-rappers-of-all-time'
page_downs = 20
api = genius.Genius(genius_client_access_token)

# eric's creds
# spotify_client_id = '96bdc763e3b5446ebbd6c44850055bcb'
# spotify_secret_id = 'f7a5054823ed491aa1856ecdc7849062'
#jeffrey's creds
spotify_client_id = '1e482b4980804a4a95e1f213a1780763'
spotify_secret_id = 'd624a1a562294382b9af1de69983c452'
spotify_token = 'AQCggMnntrbJH6DdrP5uOZqDz8tgTCj2yNg4btlRC2CSmC4h4pOGltElI1PvU6DVpx2J-suPVUUuBa0VyZwu_kejvUeNt9Xv3DfiGnRvrpuE9-7vyfziBLUKcRQsih-wTzGAmakFdWrUMPCrwwGoVVoT8dN4LI0iqZF1dDAbERumRMtZOUU87zwF24_1KPlWOKLj28A'


In [136]:
def get_artists(url, page_downs, driver_path):
    driver = webdriver.Chrome(driver_path)
    driver.get(url)
    time.sleep(1)
    page = driver.find_element_by_tag_name('body')
    for i in range(page_downs):
        page.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
    artists = driver.find_elements_by_class_name('listItem__data')
    names = [artist.find_element_by_class_name('listItem__title').text for artist in artists[:-1]]
    driver.quit()
    
    return names

In [151]:
def get_songs(artist):
    return api.search_artist(artist, verbose=False, max_songs=10)

In [152]:
artists = get_artists(artist_url, page_downs, driver_path)

KeyboardInterrupt: 

In [149]:
print(artists)

['Tupac', 'Eminem', 'The Notorious B.I.G.', 'Nas', 'Ice Cube', 'Jay-Z', 'Snoop Dogg', 'Dr. Dre', 'Kendrick Lamar', 'Rakim', 'André 3000', 'Eazy-E', '50 Cent', 'Kanye West', 'DMX', 'J. Cole', 'Busta Rhymes', 'Method Man', 'Mos Def', 'Lil Wayne', 'Ludacris', 'KRS-One', 'LL Cool J', 'Common', 'Ghostface Killah', 'Big L', 'Redman', 'T.I.', 'Big Pun', 'Nate Dogg', 'Tech N9ne', 'Lauryn Hill', 'Scarface', 'Slick Rick', 'The Game', 'Raekwon', 'Big Daddy Kane', "Ol' Dirty Bastard", 'Mobb Deep', 'Chance the Rapper', 'Logic', 'Cypress Hill', 'Ice-T', 'Lupe Fiasco', 'RZA', 'GZA', 'Warren G', 'Q-Tip', 'ASAP Rocky', 'Xzibit', 'Talib Kweli', 'Big Sean', 'Joey Badass', 'Missy Elliott', 'Drake', 'Kid Cudi', 'Donald Glover', 'Immortal Technique', 'Twista', 'Chuck D', 'Big Boi', 'Eric B. & Rakim', 'Wiz Khalifa', 'Schoolboy Q', 'Tyler, the Creator', 'Hopsin', 'Nelly', 'DMC', 'Royce da 5\'9"', 'D12', 'Jadakiss', 'Kurupt', 'Grandmaster Flash and the Furious Five', 'MC Ren', 'Gang Starr', 'Too $hort', 'E-40'

In [154]:
# test set of artists
artists = ['Kanye West', 'Migos']

In [155]:
music = []
for artist in artists:
    try:
        music = music + get_songs(artist).songs
    except AssertionError:
        pass

In [156]:
print(music)

[("'03 Electric Relaxation", 'Kanye West'), ('18 Years', 'Kanye West'), ('1996 Fat Beats Freestyle', 'Kanye West'), ('2002 Unreleased Freestyle', 'Kanye West'), ('2004 Tim Westwood Freestyle', 'Kanye West'), ('2013 Yeezus Tour Dates', 'Kanye West'), ('2013 Zane Lowe BBC Radio Interview Part 1', 'Kanye West'), ('2013 Zane Lowe BBC Radio Interview Part 2', 'Kanye West'), ('2014 Yeezus Tour Dates', 'Kanye West'), ('2015 BET Honors Visionary Award Acceptance Speech', 'Kanye West'), ('1017', 'Migos'), ('11 Birds', 'Migos'), ('2Pac & Biggie', 'Migos'), ('3 Mics', 'Migos'), ('3 Way EP [Album Art + Tracklist]', 'Migos'), ('3 Way (Intro)', 'Migos'), ('50 Bitches (Lick It Off)', 'Migos'), ('50 Chicks', 'Migos'), ('9 On Me', 'Migos'), ('Add It Up', 'Migos')]


In [24]:
df = pd.DataFrame()
df['artist'] = list(map(lambda x: x.artist, music))
df['song'] = list(map(lambda x: x.title, music))
df['album'] = list(map(lambda x: x.album, music))
df['lyrics'] = list(map(lambda x: x.lyrics, music))
df.head()

,artist,song,album,lyrics
0,Kanye West,'03 Electric Relaxation,None,You got to\n\nRelax yourself or we'll whack ya...
1,Kanye West,18 Years,Def Poetry Jam,"The next guest.. is my man\nBringing him back,..."
2,Kanye West,1996 Fat Beats Freestyle,None,"Yo it's the Lake Shore Drive\nKeep it live, ti..."
3,Kanye West,2002 Unreleased Freestyle,None,"Ayo\nPlanes, trains, automobiles\nThe G500s wi..."
4,Kanye West,2004 Tim Westwood Freestyle,None,I told her to drive over after dude crib\nBrin...


In [163]:
# list(map(lambda x: x.annotation, music))
# print(type(music[0]))
# print(music[0].__dict__['_body'])

import requests
from bs4 import BeautifulSoup
import json

def get_annotations(song):
    print(song.title)
    api_calls = [annotation['api_path'] for annotation in
                 song.__dict__['_body']['description_annotation']['annotations']]
#     print(song.__dict__.keys())
#     print(song.__dict__)
#     print(song.__dict__['_body'])
    #     print(api_calls)
    def get_from_api(api_url):
        print('http://api.genius.com' + api_url)
        page = requests.get('http://api.genius.com' + api_url,
                            headers={'Authorization': 'Bearer ' + genius_client_access_token})
        print(page.__dict__)
#         print(page.__dict__.keys())
    #         annotation_dict = json.loads(page.__dict__['_content'].decode())['response']['annotation']['body']['dom']
    # #         print(annotation_dict)
    #         for key in annotation_dict:
    #             print("NEW")
    #             print(key)
    #             print(annotation_dict[key])
    return [get_from_api(url) for url in api_calls]
get_annotations(music[2])


1996 Fat Beats Freestyle
http://api.genius.com/annotations/3615761
{'encoding': 'utf-8', 'history': [<Response [301]>], 'raw': <urllib3.response.HTTPResponse object at 0x7f0b6db5eb00>, 'cookies': <RequestsCookieJar[]>, 'reason': 'OK', 'url': 'https://api.genius.com/annotations/3615761', 'status_code': 200, '_content_consumed': True, 'elapsed': datetime.timedelta(0, 0, 230939), 'connection': <requests.adapters.HTTPAdapter object at 0x7f0b6e4c5940>, '_next': None, 'request': <PreparedRequest [GET]>, '_content': b'{"meta":{"status":200},"response":{"annotation":{"api_path":"/annotations/3615761","body":{"dom":{"tag":"root","children":[{"tag":"p","children":["NYC record store Fat Beats relocated in 1996 and celebrated their move by hosting a cypher including the likes of ",{"tag":"a","attributes":{"href":"https://genius.com/artists/Ill-bill"},"data":{"api_path":"/artists/1780"},"children":["Ill Bill"]},", ",{"tag":"a","attributes":{"href":"https://genius.com/artists/Lord-finesse"},"data":{

[None]

In [26]:
# headers = {'Authorization': 'Bearer %s' % genius_client_access_token}
# search_url = base_url + '/search'
# song_title = 'Mercy'
# params = {'q': song_title}
# response = requests.get(search_url, params=params, headers=headers)
# json = response.json()
# print(json)

In [27]:
df.to_pickle('data.pkl')

In [46]:
# spotify_token = 'BQCVEzRlbQgkEb06zkDYJKDQAzzGCm19cyf5pugp5RnUxls0OyJR-vlNyxa8bjIx-_vaBkXrY7FrSxMQCGsGieSXVgE8VnVlCtiNm-nNSnRDNsayJWUaMgKQwAk4iNisv_i6vx4FzpE'
spotify_token = 'BQBippMPZbTosK050_kzP3uESjQxmMuaqRYeHaswETEjxl23bOoviQ2bbuID1WVUtN0oQ55o4tXGd9FH_9JfmQ6B-Hwysq_sycqrUfKPvYnC_cBPa845hzWbiJ3E6oN8iIBSUswuPWI'
spotify = spotipy.Spotify(auth=spotify_token)
def get_song(song):
    
spotify_artists = [spotify.search(q=song.title, type='track') for song in music]

In [80]:
print(music[1])
print(music[1].title)
# print(spotify_artists[1])
print(spotify.search(q='name:Bohemian+Rhapsody', type='track'))
print

"18 Years" by Kanye West:
    The next guest.. is my man
    Bringing him back, its a pleasure to have him here
    Please give it for my ...
18 Years


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/search?type=track&limit=10&q=name%3ABohemian%2BRhapsody&offset=0:
 The access token expired

In [ ]:
yeezy = spotify.artist_top_tracks('5K4W6rqBFWDnAN6FQUkS6x')
for track in yeezy['tracks'][:10]:
    print('track    : ' + track['name'])